In [1]:
from PIL import Image 
from IPython.display import display 
import random
import json

In [2]:
# Each image is made up a series of traits
# The weightings for each trait drive the rarity and add up to 100%

Background = ["Blue","Cyan", "Orange", "Pink", "Purple", "Red", "Yellow"] 
background_weights = [16, 8, 15, 13, 17, 10, 21]

Base = ["Alien", "Ghost", "Normal", "Zombie"] 
base_weights = [8, 17, 45, 30]

Eyewear = ["Black Aviator Sunglasses", "Squared Gold Glasses", "Monocle", "None", "Thug Life Glasses", "Rounded Yellow Shades"] 
eyewear_weights = [18, 22, 5, 26, 10, 19]

Headgear= ["Bunny Ears", "Cowboy Hat", "Fedora", "Graduation Hat", "None", "Pirate Hat", "Mexican Sombrero Hat", "Top Hat", "Witch Hat"]
headgear_weights= [6, 8, 15, 5, 18, 12, 7, 16, 13]

Mouth= ["Frown", "Neutral", "Smile"]
mouth_weights=[30, 10, 60]


# Dictionary variable for each trait. 
# Eech trait corresponds to its file name

background_files = {
    "Blue":"blue", 
    "Cyan": "cyan",
    "Orange":"orange", 
    "Pink":"pink", 
    "Purple":"purple", 
    "Red":"red", 
    "Yellow":"yellow"
}

base_files = {
    "Alien":"alien",
    "Ghost":"ghost",
    "Normal":"normal",
    "Zombie":"zombie"  
}

eyewear_files = {
    "Black Aviator Sunglasses":"aviator glasses", 
    "Squared Gold Glasses":"blue gold", 
    "Monocle":"monocle",
    "None":"none",
    "Thug Life Glasses":"thuglife glass",
    "Rounded Yellow Shades":"yellow shades"
}

headgear_files = {
    "Bunny Ears":"bunny ears",
    "Cowboy Hat": "cowboy hat",
    "Fedora":"fedora",
    "Graduation Hat": "graduation hat",
    "None":"none", 
    "Pirate Hat":"pirate hat",
    "Mexican Sombrero Hat": "sombrero hat",
    "Top Hat":"top hat",
    "Witch Hat":"witch hat"
}

mouth_files = {
    "Frown":"frown", 
    "Neutral":"neutral", 
    "Smile":"smile"
}

In [3]:
## Generate Traits

TOTAL_IMAGES = 3400 # Number of random unique images we want to generate

all_images = []

# A recursive function to generate unique image combinations
def create_new_image():
    
    new_image = {} 

    # For each trait category, select a random trait based on the weightings 
    new_image ["Background"] = random.choices(Background, weights=background_weights)[0]
    new_image ["Base"] = random.choices(Base, weights= base_weights)[0]
    new_image ["Eyewear"] = random.choices(Eyewear, weights= eyewear_weights)[0]
    new_image ["Headgear"] = random.choices(Headgear, weights= headgear_weights)[0]
    new_image ["Mouth"] = random.choices(Mouth, weights= mouth_weights)[0]
    
    if new_image in all_images:
        return create_new_image()
    else:
        return new_image
    
    
# Generate the unique combinations based on trait weightings
for i in range(TOTAL_IMAGES): 
    
    new_trait_image = create_new_image()
    
    all_images.append(new_trait_image)
    


In [4]:
# Returns true if all images are unique
def all_images_unique(all_images):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images)

print("Are all images unique?", all_images_unique(all_images))

Are all images unique? True


In [5]:
# Add token Id to each image
i = 1
for item in all_images:
    item["tokenId"] = i
    i = i + 1

In [6]:
# Get Trait Counts

background_count = {}
for item in Background:
    background_count[item] = 0
    
base_count = {}
for item in Base:
    base_count[item] = 0

eyewear_count = {}
for item in Eyewear:
    eyewear_count[item] = 0

headgear_count = {}
for item in Headgear:
    headgear_count[item] = 0

mouth_count = {}
for item in Mouth:
    mouth_count[item] = 0


for image in all_images:
    background_count[image["Background"]] += 1
    base_count[image["Base"]] += 1
    eyewear_count[image["Eyewear"]] += 1
    headgear_count[image["Headgear"]] += 1
    mouth_count[image["Mouth"]] += 1

#file handling for trait count  
                                 
with open('trait_count.txt','w') as f:
    for i in [background_count,base_count, eyewear_count,headgear_count,mouth_count]:
        json.dump(i,f)
        f.write('\n')
    f.close()

In [7]:
  
    
#### Generate Images    
for item in all_images:

    background = background_files[item["Background"]]
    im1 = Image.open(r"C:\Users\theja\Documents\NFT image generator\trait-layers\backgrounds\{background}.png".format(background=background))
    base = base_files[item["Base"]]
    im2 = Image.open(r"C:\Users\theja\Documents\NFT image generator\trait-layers\base\{base}.png".format(base=base))
    eyewear = eyewear_files[item["Eyewear"]]
    im3 = Image.open(r"C:\Users\theja\Documents\NFT image generator\trait-layers\eyewear\{eyewear}.png".format(eyewear=eyewear))
    headgear = headgear_files[item["Headgear"]]
    im4 = Image.open(r"C:\Users\theja\Documents\NFT image generator\trait-layers\headgear\{headgear}.png".format(headgear=headgear))
    mouth = mouth_files[item["Mouth"]]
    im5 = Image.open(r"C:\Users\theja\Documents\NFT image generator\trait-layers\mouth\{mouth}.png".format(mouth=mouth))

    #Create each composite
    com1 = Image.alpha_composite(im1, im2)
    com2 = Image.alpha_composite(com1, im3)
    com3 = Image.alpha_composite(com2, im4)
    com4 = Image.alpha_composite(com3, im5)

    #saving the image
   
    file_name = str(item["tokenId"]) + ".png"
    com4.save("./images/" + file_name)

    #adding name to item
    Project_name= "PolyMonkeys"
    token_id = item['tokenId']
    item["Name"] = Project_name + ' #' + str(token_id)
    item["File_path"] = file_name

In [8]:
#### Generate Metadata for all Traits 
METADATA_FILE_NAME = './metadata/all-traits.json'; 
with open(METADATA_FILE_NAME, 'w') as outfile:
    json.dump(all_images, outfile, indent=4)
###metadata textfile
with open('imagelist.json', 'w') as f:
    json.dump(all_images, f)

#### Generate Metadata for each Image    

f = open('./metadata/all-traits.json',) 
data = json.load(f)

PROJECT_NAME = "PolyMonkeys"

def getAttribute(key, value):
    return {
        "trait_type": key,
        "value": value
    }
for i in data:
    token_id = i['tokenId']
    token = {
        "image": str(token_id) + '.png',
        "tokenId": token_id,
        "name": PROJECT_NAME + ' ' + str(token_id),
        "attributes": []
    }
    token["attributes"].append(getAttribute("Background", i["Background"]))
    token["attributes"].append(getAttribute("Base", i["Base"]))
    token["attributes"].append(getAttribute("Eyewear", i["Eyewear"]))
    token["attributes"].append(getAttribute("Headgear", i["Headgear"]))
    token["attributes"].append(getAttribute("Mouth", i["Mouth"]))


    with open('./metadata/' + str(token_id), 'w') as outfile:
        json.dump(token, outfile, indent=4)
f.close()